## Imports and Set Up

In [194]:
import os
import random
import warnings
from concurrent.futures import ThreadPoolExecutor

from scipy.optimize import minimize

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from colorama import Fore, Style
from IPython.display import clear_output
from lightgbm import LGBMRegressor
from matplotlib import pyplot as plt
from sklearn.base import clone
from sklearn.ensemble import VotingRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import (accuracy_score, cohen_kappa_score,
                             confusion_matrix, f1_score, mean_absolute_error,
                             mean_squared_error, precision_score, recall_score,
                             classification_report)
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [195]:
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [196]:
def set_seed(seed_value=2024):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(2024)

## Data Processing

### Load in Files

In [197]:
file_path = '/Users/naliniramanathan/projects/ml_course/final_project/kaggle/input/cmi-piu'

In [198]:
TRAIN_CSV = f'{file_path}/train.csv'
TEST_CSV = f'{file_path}/test.csv'
SAMPLE_SUBMISSION_CSV = f'{file_path}/sample_submission.csv'
SERIES_TRAIN_DIR = f'{file_path}/series_train.parquet'
SERIES_TEST_DIR = f'{file_path}/series_test.parquet'

train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
sample_submission_df = pd.read_csv(SAMPLE_SUBMISSION_CSV)

# Drop all the PCIAT variables as they are not present in the test data
for col in train_df.columns:
    if 'PCIAT' in col:
        train_df.drop(col, axis=1, inplace=True)

In [199]:
# Function to process individual time series files
def process_time_series(file_name, directory):
    df = pd.read_parquet(os.path.join(directory, file_name, 'part-0.parquet'))
    df = df.drop('step', axis=1)
    stats = df.describe().values.flatten()
    record_id = file_name.split('=')[1]
    return stats, record_id

In [200]:
# Function to load and aggregate time series data
def load_time_series_data(directory):
    file_names = os.listdir(directory)
    stats_list = []
    ids_list = []

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_time_series(fname, directory), file_names),
                            total=len(file_names)))

    for stats, record_id in results:
        stats_list.append(stats)
        ids_list.append(record_id)

    stats_df = pd.DataFrame(stats_list, columns=[f'stat_{i}' for i in range(len(stats_list[0]))])
    stats_df['id'] = ids_list
    return stats_df

In [201]:
train_series_df = load_time_series_data(SERIES_TRAIN_DIR)
test_series_df = load_time_series_data(SERIES_TEST_DIR)

100%|██████████| 2/2 [00:00<00:00, 15.78it/s]


### Encoding of Time Series Data

In [202]:
class TimeSeriesAutoencoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(TimeSeriesAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, encoding_size * 3),
            nn.ReLU(),
            nn.Linear(encoding_size * 3, encoding_size * 2),
            nn.ReLU(),
            nn.Linear(encoding_size * 2, encoding_size),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_size, input_size * 2),
            nn.ReLU(),
            nn.Linear(input_size * 2, input_size * 3),
            nn.ReLU(),
            nn.Linear(input_size * 3, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [203]:
def get_encoded_features(df, encoding_dim=60, epochs=100, batch_size=32):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)
    tensor_data = torch.FloatTensor(scaled_data)
    input_dim = tensor_data.shape[1]

    autoencoder = TimeSeriesAutoencoder(input_dim, encoding_dim)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())

    for epoch in range(epochs):
        for i in range(0, len(tensor_data), batch_size):
            batch = tensor_data[i:i + batch_size]
            optimizer.zero_grad()
            outputs = autoencoder(batch)
            loss = criterion(outputs, batch)
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

    with torch.no_grad():
        encoded_data = autoencoder.encoder(tensor_data).numpy()

    encoded_df = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    return encoded_df

In [204]:
train_series_features = train_series_df.drop('id', axis=1)
test_series_features = test_series_df.drop('id', axis=1)

train_encoded = get_encoded_features(train_series_features)
test_encoded = get_encoded_features(test_series_features)

train_encoded['id'] = train_series_df['id']
test_encoded['id'] = test_series_df['id']

train_df = train_df.merge(train_encoded, on='id', how='left')
test_df = test_df.merge(test_encoded, on='id', how='left')

Epoch [10/100], Loss: 0.6314
Epoch [20/100], Loss: 0.5333
Epoch [30/100], Loss: 0.4877
Epoch [40/100], Loss: 0.4503
Epoch [50/100], Loss: 0.4440
Epoch [60/100], Loss: 0.4418
Epoch [70/100], Loss: 0.4358
Epoch [80/100], Loss: 0.4334
Epoch [90/100], Loss: 0.4339
Epoch [100/100], Loss: 0.4304
Epoch [10/100], Loss: 1.0070
Epoch [20/100], Loss: 0.5783
Epoch [30/100], Loss: 0.4271
Epoch [40/100], Loss: 0.4271
Epoch [50/100], Loss: 0.4271
Epoch [60/100], Loss: 0.4271
Epoch [70/100], Loss: 0.4271
Epoch [80/100], Loss: 0.4271
Epoch [90/100], Loss: 0.4271
Epoch [100/100], Loss: 0.4271


### Imputation of Missing Numerical Values

In [205]:
# Imputing missing values using KNN imputer
def impute_missing_values(df):
    imputer = KNNImputer(n_neighbors=5)
    numeric_columns = df.select_dtypes(include=['float64','float32','int64']).columns # We treat the time series data as a median for now but others could be used
    imputed_array = imputer.fit_transform(df[numeric_columns])
    imputed_df = pd.DataFrame(imputed_array, columns=numeric_columns)
    for col in df.columns:
        if col not in numeric_columns:
            imputed_df[col] = df[col]
    return imputed_df

In [206]:
train_df.replace([np.inf, -np.inf], np.nan, inplace=True) # Debug why we have inf values

train_df = impute_missing_values(train_df)
train_df['sii'] = train_df['sii'].round().astype(int)

# Imputation is needed in test set for some cases but not others to revisit

print(train_df.isna().sum())

Basic_Demos-Age           0
Basic_Demos-Sex           0
CGAS-CGAS_Score           0
Physical-BMI              0
Physical-Height           0
                       ... 
BIA-Season             1815
PAQ_A-Season           3485
PAQ_C-Season           2239
SDS-Season             1342
PreInt_EduHx-Season     420
Length: 120, dtype: int64


### Categorical Processing

In [207]:
categorical_cols = list(train_df.select_dtypes(include=['object']).columns) #sii (outcome var) is categorical but we are encoding that differently
categorical_cols.remove('id')
print(categorical_cols)

def preprocess_categorical(df):
    for col in categorical_cols:
        df[col] = df[col].fillna('Missing').astype('category')
    return df

train_df = preprocess_categorical(train_df)
test_df = preprocess_categorical(test_df)


train_df = pd.get_dummies(train_df, columns = categorical_cols, drop_first=True, dtype='int')
test_df = pd.get_dummies(test_df, columns = categorical_cols, drop_first=True, dtype='int')

['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']


In [208]:
# # Feature engineering
# def add_engineered_features(df):
#     df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
#     df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
#     df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
#     df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
#     df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
#     df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
#     df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
#     df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
#     df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
#     df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
#     df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
#     df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
#     df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
#     df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
#     df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
#     return df

In [209]:
# train_df = add_engineered_features(train_df)
# test_df = add_engineered_features(test_df)

In [210]:
train_df = train_df.drop('id', axis=1)
test_df = test_df.drop('id', axis=1)

In [211]:
# encoded_feature_cols = [col for col in train_encoded.columns if col != 'id']
# selected_features = [
#     'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI',
#     'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#     'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#     'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
#     'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
#     'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
#     'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone',
#     'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
#     'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#     'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#     'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#     'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#     'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total',
#     'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
#     'PreInt_EduHx-computerinternet_hoursday', 'BMI_Age', 'Internet_Hours_Age',
#     'BMI_Internet_Hours', 'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW',
#     'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight', 'SMM_Height',
#     'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW'
# ] + encoded_feature_cols

In [212]:
# train_df = train_df[selected_features + ['sii']].dropna(subset=['sii'])
# test_df = test_df[selected_features]

## Model 1 - LGBM, XGBoost, CatBoost

In [213]:
def quadratic_weighted_kappa(y_actual, y_predicted):
    return cohen_kappa_score(y_actual, y_predicted, weights='quadratic')

def apply_thresholds(predictions, thresholds):
    return np.where(predictions < thresholds[0], 0,
                    np.where(predictions < thresholds[1], 1,
                             np.where(predictions < thresholds[2], 2, 3)))

def optimize_thresholds(y_true, predictions):
    def loss_func(thresh):
        return -quadratic_weighted_kappa(y_true, apply_thresholds(predictions, thresh))
    initial_thresholds = [0.5, 1.5, 2.5]
    result = minimize(loss_func, initial_thresholds, method='Nelder-Mead')
    return result.x

In [214]:
def train_and_evaluate(model):
    X = train_df.drop('sii', axis=1)
    y = train_df['sii'].astype(int)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    oof_predictions = np.zeros(len(y))
    # test_predictions = np.zeros(len(test_features))
    train_kappas = []
    val_kappas = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        print(f"Training fold {fold + 1}")
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        print(np.any(np.isnan(X_train)), np.any(np.isnan(y_train)))  # Should be False
        print(np.any(np.isinf(X_train)), np.any(np.isinf(y_train)))  # Should be False

        print(np.any(np.isnan(X_val)), np.any(np.isnan(y_val)))  # Should be False
        print(np.any(np.isinf(X_val)), np.any(np.isinf(y_val)))  # Should be False

        cloned_model = clone(model)
        cloned_model.fit(X_train, y_train)
        
        y_train_pred = cloned_model.predict(X_train).round().astype(int)
        y_val_pred = cloned_model.predict(X_val)

        oof_predictions[val_idx] = y_val_pred

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred)
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred.round().astype(int))

        train_kappas.append(train_kappa)
        val_kappas.append(val_kappa)

        # test_predictions += cloned_model.predict(test_features) / kf.n_splits

        print(f"Fold {fold + 1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Average Train QWK: {np.mean(train_kappas):.4f}")
    print(f"Average Validation QWK: {np.mean(val_kappas):.4f}")

    optimal_thresholds = optimize_thresholds(y, oof_predictions)
    print(f"Optimized Thresholds: {optimal_thresholds}")

    final_oof_predictions = apply_thresholds(oof_predictions, optimal_thresholds)
    # final_test_predictions = apply_thresholds(test_predictions, optimal_thresholds)

    final_kappa = quadratic_weighted_kappa(y, final_oof_predictions)
    print(f"Final Optimized QWK: {Fore.CYAN}{Style.BRIGHT}{final_kappa:.4f}{Style.RESET_ALL}")

In [215]:
lightgbm_params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,
    'lambda_l2': 0.01,
    'n_estimators': 300,
    'random_state': 42,
    'verbose': -1
}

xgboost_params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': 42,
    'verbosity': 0
}

# xgboost_params = {
#     'learning_rate': 0.05,
#     'max_depth': 6,
#     'n_estimators': 200,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'reg_alpha': 1,
#     'reg_lambda': 5,
#     'random_state': 42,
#     'tree_method': 'gpu_hist',
#     'verbosity': 0
# }

catboost_params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': 42,
    'verbose': 0,
    'l2_leaf_reg': 10
}


# catboost_params = {
#     'learning_rate': 0.05,
#     'depth': 6,
#     'iterations': 200,
#     'random_seed': 42,
#     'verbose': 0,
#     'l2_leaf_reg': 10,
#     'task_type': 'GPU'
# }


In [216]:
lgb_model = LGBMRegressor(**lightgbm_params)
xgb_model = XGBRegressor(**xgboost_params)
cat_model = CatBoostRegressor(**catboost_params)
ensemble_model = VotingRegressor(
    estimators=[('lgb', lgb_model), ('xgb', xgb_model), ('cat', cat_model)],
    weights=[4.0, 4.0, 5.0]
)

In [217]:
train_and_evaluate(ensemble_model)

Average Train QWK: 0.8025
Average Validation QWK: 0.4512
Optimized Thresholds: [0.55207857 1.03488576 2.70780292]
Final Optimized QWK: 0.5016
